In [1]:
# default_exp tictactoe_pyteal


In [2]:
%pip install pyteal | grep -v 'already satisfied'


Note: you may need to restart the kernel to use updated packages.


# imports

In [3]:
# export

from pyteal import *


In [4]:
import sys
sys.path.append('..')
import lib_py.util as util
import lib_py.ipython_magic as ipython_magic

import lib_py.tictactoe_pyteal_spi as tictactoe_pyteal_spi


main.spi Loaded


In [5]:
%%spi

open console_fsx
open pyteal

<magic:f980a61a3f3b4f45afaa6b18129b6b48>


<IPython.core.display.Javascript object>

'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

# numGlobal

In [6]:
%%node export

export var numGlobalByteSlices = 5
export var numGlobalInts = 4


'/workspaces/spiral-notebook/lib_ts/tictactoe_pyteal.ts'

# AppVariables, DefaultValues, AppActions

In [7]:
# export

class AppVariables:
    """
    All the variables available in the global state of the application.
    """
    PlayerXState = Bytes("PlayerXState")
    PlayerOState = Bytes("PlayerOState")

    PlayerOAddress = Bytes("PlayerOAddress")
    PlayerXAddress = Bytes("PlayerXAddress")
    PlayerTurnAddress = Bytes("PlayerTurnAddress")
    FundsEscrowAddress = Bytes("FundsEscrowAddress")

    BetAmount = Bytes("BetAmount")
    ActionTimeout = Bytes("ActionTimeout")
    GameStatus = Bytes("GameState")


class DefaultValues:
    """
    The default values for the global variables initialized on the transaction that creates the application.
    """
    PlayerXState = Int(0)
    PlayerOState = Int(0)
    GameStatus = Int(0)
    BetAmount = Int(1000000)
    GameDurationInSeconds = Int(3600)


class AppActions:
    """
    Available actions in the tic-tac-toe application.
    """
    SetupPlayers = Bytes("SetupPlayers")
    ActionMove = Bytes("ActionMove")
    MoneyRefund = Bytes("MoneyRefund")


In [8]:
%%spi

let get_bytes () =
    {
        player_x_state = new_bytes "PlayerXState"
        player_o_state = new_bytes "PlayerOState"
        player_x_address = new_bytes "PlayerXAddress"
        player_o_address = new_bytes "PlayerOAddress"
        player_turn_address = new_bytes "PlayerTurnAddress"
        funds_escrow_address = new_bytes "FundsEscrowAddress"
        action_timeout = new_bytes "ActionTimeout"
        game_status = new_bytes "GameState"
        bet_amount = new_bytes "BetAmount"
    }

inl get_default_values () = 
    {
        player_x_state = new_int 0
        player_o_state = new_int 0
        game_status = new_int 0
        bet_amount = new_int 1000000
        game_duration_seconds = new_int 3600
    }

inl get_app_actions () =
    {
        setup_players = new_bytes "SetupPlayers"
        action_move = new_bytes "ActionMove"
        money_refund = new_bytes "MoneyRefund"
    }

'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [9]:
def test(a, b):
    if a != b:
        print('\na:\n', a)
        print('\nb:\n', b)
        assert a == b


# clear_program

In [10]:
# export

def clear_program():
    print('5:clear_program')
    return Return(Int(1))


In [11]:
%%spi

inl clear_program () =
    inspect "clear_program"
    new_return (new_int 1)

'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [12]:
clear_program_python_teal = compileTeal(clear_program(), mode=Mode.Application, version=6)


5:clear_program


In [13]:
clear_program_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow9(), mode=Mode.Application, version=6)

clear_program


In [14]:
test(clear_program_python_teal, clear_program_spiral_teal)
util.write_file('../lib_teal/clear_program.teal', clear_program_spiral_teal)
print(clear_program_spiral_teal)


#pragma version 6
int 1
return


# money_refund_logic

In [15]:
# export

def money_refund_logic():
    """
    This function handles the logic for refunding the money in case of a winner, tie or timeout termination. If the
    player whose turn it is hasn't made a move for the predefined period of time, the other player is declared as a
    winner and can withdraw the money.
    This action logic should be performed using an Atomic Transfer of 2 transactions in case of a winner or using an
    Atomic Transfer of 3 transactions in case of a tie.
    If there is a winner the Atomic Transfer should have the following 2 transactions:
    1. Application Call with the appropriate application action argument.
    2. Payment from the Escrow to the Winner Address with a amount equal to the 2 * BetAmount.
    If there is a tie the Atomic Transfer should have the following 3 transactions:
    1. Application Call with the appropriate application action argument.
    2. Payment from the Escrow to the PlayerX's Address with a amount equal to the BetAmount.
    3. Payment from the Escrow to the PlayerO's Address with a amount equal to the BetAmount.
    :return:
    """
    print('5:money_refund_logic')
    has_x_won_by_playing = App.globalGet(AppVariables.GameStatus) == Int(1)
    has_o_won_by_playing = App.globalGet(AppVariables.GameStatus) == Int(2)

    has_x_won_by_timeout = And(App.globalGet(AppVariables.GameStatus) == Int(0),
                               Global.latest_timestamp() > App.globalGet(AppVariables.ActionTimeout),
                               App.globalGet(AppVariables.PlayerTurnAddress) == App.globalGet(AppVariables.PlayerOAddress))

    has_o_won_by_timeout = And(App.globalGet(AppVariables.GameStatus) == Int(0),
                               Global.latest_timestamp() > App.globalGet(AppVariables.ActionTimeout),
                               App.globalGet(AppVariables.PlayerTurnAddress) == App.globalGet(AppVariables.PlayerXAddress))

    has_x_won = Or(has_x_won_by_playing, has_x_won_by_timeout)
    has_o_won = Or(has_o_won_by_playing, has_o_won_by_timeout)
    game_is_tie = App.globalGet(AppVariables.GameStatus) == Int(3)

    x_withdraw = Seq([
        Assert(Gtxn[1].receiver() == App.globalGet(AppVariables.PlayerXAddress)),
        Assert(Gtxn[1].amount() == Int(2) * App.globalGet(AppVariables.BetAmount)),
        App.globalPut(AppVariables.GameStatus, Int(1))
    ])

    o_withdraw = Seq([
        Assert(Gtxn[1].receiver() == App.globalGet(AppVariables.PlayerOAddress)),
        Assert(Gtxn[1].amount() == Int(2) * App.globalGet(AppVariables.BetAmount)),
        App.globalPut(AppVariables.GameStatus, Int(2))
    ])

    tie_withdraw = Seq([
        Assert(Gtxn[1].receiver() == App.globalGet(AppVariables.PlayerXAddress)),
        Assert(Gtxn[1].amount() == App.globalGet(AppVariables.BetAmount)),
        Assert(Gtxn[2].type_enum() == TxnType.Payment),
        Assert(Gtxn[2].sender() == App.globalGet(AppVariables.FundsEscrowAddress)),
        Assert(Gtxn[2].receiver() == App.globalGet(AppVariables.PlayerOAddress)),
        Assert(Gtxn[2].amount() == App.globalGet(AppVariables.BetAmount))
    ])

    return Seq([
        Assert(Gtxn[1].type_enum() == TxnType.Payment),
        Assert(Gtxn[1].sender() == App.globalGet(AppVariables.FundsEscrowAddress)),
        Cond(
            [has_x_won, x_withdraw],
            [has_o_won, o_withdraw],
            [game_is_tie, tie_withdraw]
        ),
        Return(Int(1))
    ])


In [16]:
%%spi

inl money_refund_logic () =
    inspect "money_refund_logic"

    inl bytes = get_bytes ()
    
    inl has_x_won_by_playing = app_global_get bytes.game_status = new_int 1
    inl has_x_won_by_timeout = 
        new_and3
            (app_global_get bytes.game_status = new_int 0)
            (global_latest_timestamp () > app_global_get bytes.action_timeout)
            (app_global_get bytes.player_turn_address = app_global_get bytes.player_o_address)
    inl has_x_won = new_or has_x_won_by_playing has_x_won_by_timeout
    inl x_withdraw = 
        new_seq
            ;[
                new_assert (get_gtxn_receiver (get_gtxn 1) = app_global_get bytes.player_x_address)
                new_assert (get_gtxn_amount (get_gtxn 1) = new_int 2 * app_global_get bytes.bet_amount)
                app_global_put bytes.game_status (new_int 1)
            ]

    inl has_o_won_by_playing = app_global_get bytes.game_status = new_int 2
    inl has_o_won_by_timeout = 
        new_and3
            (app_global_get bytes.game_status = new_int 0)
            (global_latest_timestamp () > app_global_get bytes.action_timeout)
            (app_global_get bytes.player_turn_address = app_global_get bytes.player_x_address)
    inl has_o_won = new_or has_o_won_by_playing has_o_won_by_timeout
    inl o_withdraw =
        new_seq
            ;[
                new_assert (get_gtxn_receiver (get_gtxn 1) = app_global_get bytes.player_o_address)
                new_assert (get_gtxn_amount (get_gtxn 1) = new_int 2 * app_global_get bytes.bet_amount)
                app_global_put bytes.game_status (new_int 2)
            ]
            
    inl game_is_tie = app_global_get bytes.game_status = new_int 3
    inl tie_withdraw =
        new_seq
            ;[
                new_assert (get_gtxn_receiver (get_gtxn 1) = app_global_get bytes.player_x_address)
                new_assert (get_gtxn_amount (get_gtxn 1) = app_global_get bytes.bet_amount)
                new_assert (get_gtxn_type_enum (get_gtxn 2) = get_txn_type Payment)
                new_assert (get_gtxn_sender (get_gtxn 2) = app_global_get bytes.funds_escrow_address)
                new_assert (get_gtxn_receiver (get_gtxn 2) = app_global_get bytes.player_o_address)
                new_assert (get_gtxn_amount (get_gtxn 2) = app_global_get bytes.bet_amount)
            ]

    new_seq
        ;[
            new_assert (get_gtxn_type_enum (get_gtxn 1) = get_txn_type Payment)
            new_assert (get_gtxn_sender (get_gtxn 1) = app_global_get bytes.funds_escrow_address)
            new_cond3
                ;[has_x_won; x_withdraw]
                ;[has_o_won; o_withdraw]
                ;[game_is_tie; tie_withdraw]
            new_return (new_int 1)
        ]


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [17]:
money_refund_logic_python_teal = compileTeal(money_refund_logic(), mode=Mode.Application, version=6)


5:money_refund_logic


In [18]:
money_refund_logic_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow8(), mode=Mode.Application, version=6)


money_refund_logic


In [19]:
test(money_refund_logic_python_teal, money_refund_logic_spiral_teal)


# game_funds_escrow

In [20]:
# export

def game_funds_escrow(app_id: int):
    print('3:game_funds_escrow')

    win_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address())
    ])

    tie_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address()),
        Assert(Gtxn[2].fee() <= Int(1000)),
        Assert(Gtxn[2].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[2].rekey_to() == Global.zero_address())
    ])

    return Seq([
        Cond(
            [Global.group_size() == Int(2), win_refund],
            [Global.group_size() == Int(3), tie_refund],
        ),
        Return(Int(1))
    ])


In [21]:
%%spi

inl game_funds_escrow app_id =
    inspect "game_funds_escrow"

    inl win_refund = new_seq ;[
        new_assert (get_gtxn_application_id (get_gtxn 0) = new_int app_id)
        new_assert (get_gtxn_fee (get_gtxn 1) <= new_int 1000)
        new_assert (get_gtxn_asset_close_to (get_gtxn 1) = global_zero_address ())
        new_assert (get_gtxn_rekey_to (get_gtxn 1) = global_zero_address ())
    ]

    inl tie_refund = new_seq ;[
        new_assert (get_gtxn_application_id (get_gtxn 0) = new_int app_id)
        new_assert (get_gtxn_fee (get_gtxn 1) <= new_int 1000)
        new_assert (get_gtxn_asset_close_to (get_gtxn 1) = global_zero_address ())
        new_assert (get_gtxn_rekey_to (get_gtxn 1) = global_zero_address ())
        new_assert (get_gtxn_fee (get_gtxn 2) <= new_int 1000)
        new_assert (get_gtxn_asset_close_to (get_gtxn 2) = global_zero_address ())
        new_assert (get_gtxn_rekey_to (get_gtxn 2) = global_zero_address ())
    ]

    new_seq ;[
        new_cond
            ;[global_group_size () = new_int 2; win_refund]
            ;[global_group_size () = new_int 3; tie_refund]
        new_return (new_int 1)
    ]


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [22]:
game_funds_escrow_python_teal = compileTeal(game_funds_escrow(999), mode=Mode.Signature, version=6)


3:game_funds_escrow


In [23]:
game_funds_escrow_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow7(999), mode=Mode.Signature, version=6)


game_funds_escrow


In [24]:
test(game_funds_escrow_python_teal, game_funds_escrow_spiral_teal)
util.write_file('../lib_teal/game_funds_escrow.teal', game_funds_escrow_spiral_teal)
print(game_funds_escrow_spiral_teal)


#pragma version 6
global GroupSize
int 2
==
bnz main_l4
global GroupSize
int 3
==
bnz main_l3
err
main_l3:
gtxn 0 ApplicationID
int 999
==
assert
gtxn 1 Fee
int 1000
<=
assert
gtxn 1 AssetCloseTo
global ZeroAddress
==
assert
gtxn 1 RekeyTo
global ZeroAddress
==
assert
gtxn 2 Fee
int 1000
<=
assert
gtxn 2 AssetCloseTo
global ZeroAddress
==
assert
gtxn 2 RekeyTo
global ZeroAddress
==
assert
b main_l5
main_l4:
gtxn 0 ApplicationID
int 999
==
assert
gtxn 1 Fee
int 1000
<=
assert
gtxn 1 AssetCloseTo
global ZeroAddress
==
assert
gtxn 1 RekeyTo
global ZeroAddress
==
assert
main_l5:
int 1
return


# has_player_won

In [25]:
# export

def has_player_won(state):
    """
    Checks whether the passed state as an argument is a winning state. There are 8 possible winning states in which
    a specific pattern of bits needs to be activated.
    :param state:
    :return:
    """
    print('5:has_player_won')
    WINNING_STATES = [448, 56, 7, 292, 146, 73, 273, 84]
    return If(Or(BitwiseAnd(state, Int(WINNING_STATES[0])) == Int(WINNING_STATES[0]),
                 BitwiseAnd(state, Int(WINNING_STATES[1])) == Int(WINNING_STATES[1]),
                 BitwiseAnd(state, Int(WINNING_STATES[2])) == Int(WINNING_STATES[2]),
                 BitwiseAnd(state, Int(WINNING_STATES[3])) == Int(WINNING_STATES[3]),
                 BitwiseAnd(state, Int(WINNING_STATES[4])) == Int(WINNING_STATES[4]),
                 BitwiseAnd(state, Int(WINNING_STATES[5])) == Int(WINNING_STATES[5]),
                 BitwiseAnd(state, Int(WINNING_STATES[6])) == Int(WINNING_STATES[6]),
                 BitwiseAnd(state, Int(WINNING_STATES[7])) == Int(WINNING_STATES[7])), Int(1), Int(0))


In [26]:
%%spi

inl _get_winning_moves () =
    ;[
        ;[1; 2; 3]
        ;[4; 5; 6]
        ;[7; 8; 9]
        ;[1; 4; 7]
        ;[2; 5; 8]
        ;[3; 6; 9]
        ;[1; 5; 9]
        ;[3; 5; 7]
    ]

inl get_win_state (i : i32) : i32 =
    match i with
    | 0 => 448
    | 1 => 56
    | 2 => 7
    | 3 => 292
    | 4 => 146
    | 5 => 73
    | 6 => 273
    | 7 => 84
    |> new_int

inl has_player_won (state : i32) =
    inspect "has_player_won"

    new_if_else
        (new_or8 
            (new_bitwise_and state (get_win_state 0) = get_win_state 0)
            (new_bitwise_and state (get_win_state 1) = get_win_state 1)
            (new_bitwise_and state (get_win_state 2) = get_win_state 2)
            (new_bitwise_and state (get_win_state 3) = get_win_state 3)
            (new_bitwise_and state (get_win_state 4) = get_win_state 4)
            (new_bitwise_and state (get_win_state 5) = get_win_state 5)
            (new_bitwise_and state (get_win_state 6) = get_win_state 6)
            (new_bitwise_and state (get_win_state 7) = get_win_state 7))
        (new_int 1) 
        (new_int 0)

'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [27]:
has_player_won_python_teal = compileTeal(has_player_won(Int(999)), mode=Mode.Application, version=6)


5:has_player_won


In [28]:
has_player_won_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow6(Int(999)), mode=Mode.Application, version=6)


has_player_won


In [29]:
test(has_player_won_python_teal, has_player_won_spiral_teal)


# is_tie

In [30]:
#export

def is_tie():
    """
    Checks whether the game has ended with a tie. Tie state is represented with the number 511 which is the number where
    the first 9 bits are active.
    :return:
    """
    print('5:is_tie')
    state_x = App.globalGet(AppVariables.PlayerXState)
    state_o = App.globalGet(AppVariables.PlayerOState)
    return Int(511) == BitwiseOr(state_x, state_o)


In [31]:
%%spi

inl is_tie () =
    inspect "is_tie"

    inl bytes = get_bytes ()

    inl state_x = app_global_get bytes.player_x_state
    inl state_o = app_global_get bytes.player_o_state
    new_int 511 = new_bitwise_or state_x state_o


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [32]:
is_tie_python_teal = compileTeal(is_tie(), mode=Mode.Application, version=6)


5:is_tie


In [33]:
is_tie_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow5(), mode=Mode.Application, version=6)


is_tie


In [34]:
test(is_tie_python_teal, is_tie_spiral_teal)


# play_action_logic

In [35]:
# export

def play_action_logic():
    """
    Executes an action for the current player in the game and accordingly updates the state of the game. The action
    is passed as an argument to the application call.
    :return:
    """
    print('5:play_action_logic')
    position_index = Btoi(Txn.application_args[1])

    state_x = App.globalGet(AppVariables.PlayerXState)
    state_o = App.globalGet(AppVariables.PlayerOState)

    game_action = ShiftLeft(Int(1), position_index)

    player_x_move = Seq([
        App.globalPut(AppVariables.PlayerXState, BitwiseOr(state_x, game_action)),
        If(has_player_won(App.globalGet(AppVariables.PlayerXState)), App.globalPut(AppVariables.GameStatus, Int(1))),
        App.globalPut(AppVariables.PlayerTurnAddress, App.globalGet(AppVariables.PlayerOAddress)),
    ])

    player_o_move = Seq([
        App.globalPut(AppVariables.PlayerOState, BitwiseOr(state_o, game_action)),
        If(has_player_won(App.globalGet(AppVariables.PlayerOState)), App.globalPut(AppVariables.GameStatus, Int(2))),
        App.globalPut(AppVariables.PlayerTurnAddress, App.globalGet(AppVariables.PlayerXAddress)),
    ])

    return Seq([
        Assert(position_index >= Int(0)),
        Assert(position_index <= Int(8)),
        Assert(Global.latest_timestamp() <= App.globalGet(AppVariables.ActionTimeout)),
        Assert(App.globalGet(AppVariables.GameStatus) == DefaultValues.GameStatus),
        Assert(Txn.sender() == App.globalGet(AppVariables.PlayerTurnAddress)),
        Assert(And(BitwiseAnd(state_x, game_action) == Int(0), 
                   BitwiseAnd(state_o, game_action) == Int(0))),
        Cond(
            [Txn.sender() == App.globalGet(AppVariables.PlayerXAddress), player_x_move],
            [Txn.sender() == App.globalGet(AppVariables.PlayerOAddress), player_o_move],
        ),
        If(is_tie(), App.globalPut(AppVariables.GameStatus, Int(3))),
        Return(Int(1))
    ])


In [36]:
%%spi

inl play_action_logic () =
    inspect "play_action_logic"

    inl bytes = get_bytes ()
    inl default_values = get_default_values ()
    inl position_index = new_btoi (get_txn_application_args 1)
    inl state_x = app_global_get bytes.player_x_state
    inl state_o = app_global_get bytes.player_o_state

    inl game_action = new_shift_left (new_int 1) position_index

    inl player_x_move =
        new_seq
            ;[
                app_global_put bytes.player_x_state (new_bitwise_or state_x game_action)
                new_if 
                    (has_player_won (app_global_get bytes.player_x_state))
                    (app_global_put bytes.game_status (new_int 1))
                app_global_put bytes.player_turn_address (app_global_get bytes.player_o_address)
            ]

    inl player_o_move =
        new_seq
            ;[
                app_global_put bytes.player_o_state (new_bitwise_or state_o game_action)
                new_if 
                    (has_player_won (app_global_get bytes.player_o_state))
                    (app_global_put bytes.game_status (new_int 2))
                app_global_put bytes.player_turn_address (app_global_get bytes.player_x_address)
            ]

    new_seq
        ;[
            new_assert (position_index >= new_int 0)
            new_assert (position_index <= new_int 8)
            new_assert (global_latest_timestamp () <= app_global_get bytes.action_timeout)
            new_assert (app_global_get bytes.game_status = default_values.game_status)
            new_assert (get_txn_sender () = app_global_get bytes.player_turn_address)
            new_assert 
                (new_and 
                    (new_bitwise_and state_x game_action = new_int 0)
                    (new_bitwise_and state_o game_action = new_int 0))
            new_cond
                ;[get_txn_sender () = app_global_get bytes.player_x_address; player_x_move]
                ;[get_txn_sender () = app_global_get bytes.player_o_address; player_o_move]
            new_if 
                (is_tie ())
                (app_global_put bytes.game_status (new_int 3))
            new_return (new_int 1)
        ]


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [37]:
play_action_logic_python_teal = compileTeal(play_action_logic(), mode=Mode.Application, version=6)


5:play_action_logic
5:has_player_won
5:has_player_won
5:is_tie


In [38]:
play_action_logic_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow4(), mode=Mode.Application, version=6)


play_action_logic
has_player_won
has_player_won
is_tie


In [39]:
test(play_action_logic_python_teal, play_action_logic_spiral_teal)


# initialize_players_logic

In [40]:
# export

def initialize_players_logic():
    """
    This function initializes all the other global variables. The end of the execution of this function defines the game
    start. We expect that this logic is performed within an Atomic Transfer of 3 transactions:
    1. Application Call with the appropriate application action argument.
    2. Payment transaction from Player X that funds the Escrow account. The address of this sender is represents the
    PlayerX address.
    3. Payment transaction from Player O that funds the Escrow account. The address of this sender is represents the
    PlayerO address.
    :return:
    """
    print('5:initialize_players_logic')
    player_x_address = App.globalGetEx(Int(0), AppVariables.PlayerXAddress)
    player_o_address = App.globalGetEx(Int(0), AppVariables.PlayerOAddress)

    setup_failed = Seq([
        Return(Int(0))
    ])

    setup_players = Seq([
        Assert(Gtxn[1].type_enum() == TxnType.Payment),
        Assert(Gtxn[2].type_enum() == TxnType.Payment),
        Assert(Gtxn[1].receiver() == Gtxn[2].receiver()),
        Assert(Gtxn[1].amount() == App.globalGet(AppVariables.BetAmount)),
        Assert(Gtxn[2].amount() == App.globalGet(AppVariables.BetAmount)),
        App.globalPut(AppVariables.PlayerXAddress, Gtxn[1].sender()),
        App.globalPut(AppVariables.PlayerOAddress, Gtxn[2].sender()),
        App.globalPut(AppVariables.PlayerTurnAddress, Gtxn[1].sender()),
        App.globalPut(AppVariables.FundsEscrowAddress, Gtxn[1].receiver()),
        App.globalPut(AppVariables.ActionTimeout, Global.latest_timestamp() + DefaultValues.GameDurationInSeconds),
        Return(Int(1))
    ])

    return Seq([
        player_x_address,
        player_o_address,
        If(Or(player_x_address.hasValue(), player_o_address.hasValue()),
           setup_failed, setup_players)
    ])


In [41]:
%%spi

inl initialize_players_logic game_duration_seconds =
    inspect "initialize_players_logic"

    inl bytes = get_bytes ()
    inl default_values = get_default_values ()
    inl player_x_address = app_global_get_ex (new_int 0) bytes.player_x_address
    inl player_o_address = app_global_get_ex (new_int 0) bytes.player_o_address
    inl setup_failed = new_seq ;[new_return (new_int 0)]

    inl game_duration_int =
        match game_duration_seconds with
        | Some duration => new_int duration
        | None => default_values.game_duration_seconds
    
    inl setup_players = 
        new_seq 
            ;[
                new_assert (get_gtxn_type_enum (get_gtxn 1) = get_txn_type Payment)
                new_assert (get_gtxn_type_enum (get_gtxn 2) = get_txn_type Payment)
                new_assert (get_gtxn_receiver (get_gtxn 1) = get_gtxn_receiver (get_gtxn 2))
                new_assert (get_gtxn_amount (get_gtxn 1) = app_global_get bytes.bet_amount)
                new_assert (get_gtxn_amount (get_gtxn 2) = app_global_get bytes.bet_amount)
                app_global_put bytes.player_x_address (get_gtxn_sender (get_gtxn 1))
                app_global_put bytes.player_o_address (get_gtxn_sender (get_gtxn 2))
                app_global_put bytes.player_turn_address (get_gtxn_sender (get_gtxn 1))
                app_global_put bytes.funds_escrow_address (get_gtxn_receiver (get_gtxn 1))
                app_global_put bytes.action_timeout (global_latest_timestamp () + game_duration_int)
                new_return (new_int 1)
            ]

    new_seq3
        player_x_address
        player_o_address
        (new_if_else
            (new_or 
                (app_global_has_value player_x_address)
                (app_global_has_value player_o_address))
            setup_failed
            setup_players)


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [42]:
initialize_players_logic_python_teal = compileTeal(initialize_players_logic(), mode=Mode.Application, version=6)


5:initialize_players_logic


In [43]:
initialize_players_logic_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow3(tictactoe_pyteal_spi.US0(0)), mode=Mode.Application, version=6)


initialize_players_logic


In [44]:
test(initialize_players_logic_python_teal, initialize_players_logic_spiral_teal)


# app_initialization_logic

In [45]:
# export

def app_initialization_logic():
    """
    Initialization of the default global variables.
    """
    print('5:app_initialization_logic')
    return Seq([
        App.globalPut(AppVariables.PlayerXState, DefaultValues.PlayerXState),
        App.globalPut(AppVariables.PlayerOState, DefaultValues.PlayerOState),
        App.globalPut(AppVariables.GameStatus, DefaultValues.GameStatus),
        App.globalPut(AppVariables.BetAmount, DefaultValues.BetAmount),
        Return(Int(1))
    ])


In [46]:
%%spi

inl app_initialization_logic () =
    inspect "app_initialization_logic"

    inl default_values = get_default_values ()
    inl bytes = get_bytes ()

    new_seq 
        ;[
            app_global_put bytes.player_x_state default_values.player_x_state
            app_global_put bytes.player_o_state default_values.player_o_state
            app_global_put bytes.game_status default_values.game_status
            app_global_put bytes.bet_amount default_values.bet_amount
            new_return (new_int 1)
        ]


'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [47]:
app_initialization_logic_python_teal = compileTeal(app_initialization_logic(), mode=Mode.Application, version=6)


5:app_initialization_logic


In [48]:
app_initialization_logic_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow2(), mode=Mode.Application, version=6)


app_initialization_logic


In [49]:
test(app_initialization_logic_python_teal, app_initialization_logic_spiral_teal)


# application_start

In [50]:
# export

def application_start():
    """
    This function represents the start of the application. Here we decide which action will be executed in the current
    application call. If we are creating the application for the very first time we are going to initialize some
    of the global values with their appropriate default values.
    """
    print('5:application_start')
    is_app_initialization = Txn.application_id() == Int(0)

    actions = Cond(
        [Txn.application_args[0] == AppActions.SetupPlayers, initialize_players_logic()],
        [And(Txn.application_args[0] == AppActions.ActionMove, Global.group_size() == Int(1)), 
             play_action_logic()],
        [Txn.application_args[0] == AppActions.MoneyRefund, money_refund_logic()]
    )

    return If(is_app_initialization, app_initialization_logic(), actions)


In [51]:
%%spi

inl application_start game_duration_seconds =
    inspect "application_start"
    
    inl app_actions = get_app_actions ()

    inl is_app_initialization = get_txn_application_id () = new_int 0
    inl actions =
        new_cond3
            ;[get_txn_application_args 0 = app_actions.setup_players; initialize_players_logic game_duration_seconds]
            ;[new_and 
                (get_txn_application_args 0 = app_actions.action_move) 
                (global_group_size () = new_int 1)
              play_action_logic ()]
            ;[get_txn_application_args 0 = app_actions.money_refund; money_refund_logic ()]

    new_if_else is_app_initialization (app_initialization_logic ()) actions

'/workspaces/spiral-notebook/lib_spi/tictactoe_pyteal.spi'

In [52]:
application_start_python_teal = compileTeal(application_start(), mode=Mode.Application, version=6)


5:application_start
5:initialize_players_logic
5:play_action_logic
5:has_player_won
5:has_player_won
5:is_tie
5:money_refund_logic
5:app_initialization_logic


In [53]:
application_start_spiral_teal = compileTeal(tictactoe_pyteal_spi._arrow1(tictactoe_pyteal_spi.US0(0)), mode=Mode.Application, version=6)


application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic


In [54]:
test(application_start_python_teal, application_start_spiral_teal)
util.write_file('../lib_teal/application_start.teal', application_start_spiral_teal)
print(application_start_spiral_teal)


#pragma version 6
txn ApplicationID
int 0
==
bnz main_l34
txna ApplicationArgs 0
byte "SetupPlayers"
==
bnz main_l31
txna ApplicationArgs 0
byte "ActionMove"
==
global GroupSize
int 1
==
&&
bnz main_l13
txna ApplicationArgs 0
byte "MoneyRefund"
==
bnz main_l5
err
main_l5:
gtxn 1 TypeEnum
int pay
==
assert
gtxn 1 Sender
byte "FundsEscrowAddress"
app_global_get
==
assert
byte "GameState"
app_global_get
int 1
==
byte "GameState"
app_global_get
int 0
==
global LatestTimestamp
byte "ActionTimeout"
app_global_get
>
&&
byte "PlayerTurnAddress"
app_global_get
byte "PlayerOAddress"
app_global_get
==
&&
||
bnz main_l12
byte "GameState"
app_global_get
int 2
==
byte "GameState"
app_global_get
int 0
==
global LatestTimestamp
byte "ActionTimeout"
app_global_get
>
&&
byte "PlayerTurnAddress"
app_global_get
byte "PlayerXAddress"
app_global_get
==
&&
||
bnz main_l11
byte "GameState"
app_global_get
int 3
==
bnz main_l9
err
main_l9:
gtxn 1 Receiver
byte "PlayerXAddress"
app_global_get
==
assert
gtxn 1 Am

# main

In [55]:
%%spiral

open lib_spi.util
open lib_spi.console_fsx
open lib_spi.tictactoe_pyteal

inl imports () =
    $"#r \"nuget: Fable.Python\""
    $"open Fable.Core.PyInterop"
    $"open Fable.Core"

inl tests () =
    inl a = 1i32
    inl b = 1i32
    test (a = b)

inl main () =
    inspect "main.spi Loaded"

    imports ()
    tests ()

    (
        application_start,
        app_initialization_logic,
        initialize_players_logic,
        play_action_logic,
        is_tie,
        has_player_won,
        game_funds_escrow,
        money_refund_logic,
        clear_program
    )
    |> dyn
    |> ignore


{'run_node_output': [], 'fsx_path': '/workspaces/spiral-notebook/lib_fsx/tictactoe_pyteal_spi.fsx', 'len(new_code_fsx)': 48509, 'new_code_fsx[:100]': '#r "nuget: Fable.Python"\nopen Fable.Core.PyInterop\nopen Fable.Core\n\ntype [<Struct>] US0 =\n    | US0_'}


'/workspaces/spiral-notebook/main.spi'

In [56]:
%%node run

import * as spiral_api from "./spiral_api"

await spiral_api.spiToFsx()

[]

In [57]:
%%spiral run

inspect "test2"


{'run_node_output': [], 'fsx_path': '/workspaces/spiral-notebook/lib_fsx/_ipython_spi.fsx', 'len(new_code_fsx)': 46, 'new_code_fsx[:100]': 'let v0 : string = "test2"\nprintfn $"%A{v0}"\n()'}


'/workspaces/spiral-notebook/main.spi'